In [2]:
def read_data():
    import pandas as pd, numpy as np
    np.set_printoptions(linewidth=140)
    file = "~/dropbox/voting/germany-2021.xlsx"
    #file = "prufa.xlsx"
    df = pd.read_excel(file, index_col=0)
    votes = np.array(df.iloc[:-1, :-1])
    land_seats = np.array(df.Total[:-1]).astype(int).tolist()
    party_seats = np.array(df.loc["Total"][:-1]).astype(int).tolist()
    #display(df)
    return (votes, land_seats, party_seats)

(votes, land_seats, party_seats) = read_data()
sum(party_seats), sum(land_seats)

(598, 598)

In [3]:
land_seats, party_seats

([22, 13, 59, 5, 127, 43, 30, 77, 93, 7, 24, 20, 13, 32, 17, 16],
 [126, 168, 70, 77, 31, 95, 30, 1, 0])

In [4]:
import numpy as np
def gurobi_max(votes, const_seats, party_seats, div):
    import gurobipy as gp
    (NC, NP) = votes.shape
    P = range(NP)
    C = range(NC)
    S = [range(const_seats[c]) for c in C]
    m = gp.Model()
    x = []
    NP = len(party_seats)
    for c in C:
        x.append(m.addMVar((NP, const_seats[c]), vtype='B'))
    n = m.addMVar(votes.shape, vtype=gp.GRB.INTEGER)
    m.addConstrs((n[c,p] == sum(x[c][p,:]) for c in C for p in P))
    m.addConstrs((sum(n[:,p]) <= party_seats[p] for p in P))
    m.addConstrs((sum(n[c,:]) == const_seats[c] for c in C))
    obj = gp.LinExpr()
    for c in C:
        for s in S[c]:
            a = np.log(np.maximum(1,votes[c,:])/div[s])
            obj.add(gp.LinExpr((a[p], x[c][p,s]) for p in P))
    m.setObjective(obj, sense=gp.GRB.MAXIMIZE)
    m.setParam( 'OutputFlag', False)
    m.optimize()
    n = n.X.astype(int)
    return n

SL_div = np.arange(1, 2*max(party_seats)+1, 2)
half_seats = [l//2 for l in land_seats]
lp_seats = gurobi_max(votes, land_seats, party_seats, SL_div)
lp_nat_seats = gurobi_max(votes, half_seats, party_seats, SL_div)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-09-29


In [5]:
%timeit -n20 -r5 gurobi_max(votes, land_seats, party_seats, SL_div)

47.2 ms ± 4.25 ms per loop (mean ± std. dev. of 5 runs, 20 loops each)


In [6]:
def entropy(votes, seats, div):
    (m,n) = votes.shape
    C = range(m)
    P = range(n)
    summa = 0
    for c in C:
        for p in P:
            for s in range(seats[c,p]):
                summa += np.log(np.maximum(1, votes[c,p])/div[s])
    return summa

In [18]:
import sys
sys.path.append('/Users/jonasson/voting/backend')
sys.path.append('/Users/jonasson/voting/backend/methods')
from alternating_scaling import alt_scaling
from division_rules import sainte_lague_gen
print(2+2)

def altscaling(votes, land_seats, party_seats, nat_seats):
    prior_alloc = np.zeros(votes.shape, int)
    seats, _ = alt_scaling(votes, land_seats, party_seats, prior_alloc,
                           sainte_lague_gen, None)
    return np.array(seats)

nat_seats = sum(land_seats) - sum(half_seats)
altscal_seats = altscaling(votes, land_seats, party_seats, 0)
altscal_nat_seats = altscaling(votes, half_seats, party_seats, nat_seats)

4


KeyboardInterrupt: 

In [ ]:
%timeit -n20 -r5 altscaling(votes, land_seats, party_seats)

In [117]:
print(sum(lp_seats))
print(sum(altscal_seats))
print(entropy(votes, altscal_seats, SL_div))
print(entropy(votes, lp_seats, SL_div))
print(entropy(votes, altscal_nat_seats, SL_div))
print(entropy(votes, lp_nat_seats, SL_div))

[126 168  70  77  31  95  30   1   0]
[126 168  70  77  31  95  30   1   0]
6828.838634270492
6828.838634270492
3549.0747124969107
3549.0747124969107
